In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore')
import pandas as pd
import platform
import matplotlib
import matplotlib.pyplot as plt
if platform.system() == 'Windows':
    matplotlib.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    matplotlib.rc('font', family='AppleGothic')
else:
    matplotlib.rc('font', family='NanumGothic')
    
# tensorflow warning off
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)    

In [2]:
df = pd.read_csv('input_data.csv',  encoding='utf-8')

In [3]:
teamids=['HH','HT','KT','LG','LT','NC','OB','SK','SS','WO',]
features=list(df)
features=features[5:]
str(features)

"['경기수', '피장타율', '방어율', '삼진/볼넷', '피출루율', '피OPS', '9이닝당 삼진수', '9이닝당 볼넷수', '잔루처리율', '해당경기 실점', 'cOBP', 'cSLG', 'cOPS', 'cBABIP', 'KK/BB', 'wOBA', 'WLD', 'PARK', 'BPF', 'PPF', '피장타율_', '방어율_', '삼진/볼넷_', '피출루율_', '피OPS_', '9이닝당삼진수_', '9이닝당볼넷수_', '잔루처리율_', 'RUN']"

In [4]:
def make_sum(df,S,E):  # dictionary 형태의 dataset을 형성한다.
    dic = {}
    for i in range(S,E):
        tid = df['T_ID'][i]
        vstid = df['VS_T_ID'][i]
        for j in features:
            globals()['{}'.format(j)] =df[j][i]
        # 팀 아이디가 딕셔너리에 없을 경우
        if df['T_ID'][i] not in dic.keys():
            dic[tid] = {}

        # 상대 팀 아이디가 딕셔너리에 없을 경우
        if df['VS_T_ID'][i] not in dic[tid].keys():
            dic[tid][vstid] = {}
            dic[tid][vstid] = [1]
            for j in range(len(features)):
                dic[tid][vstid].append(df[features[j]][i])

        else:
            dic[tid][vstid][0]+=1  #tid랑 vstid랑 잔여경기중에서 몇번 만났는지
            for j in range(len(features)):
                dic[tid][vstid][j+1]+=df[features[j]][i] #각 feature들을 더해줌
    return dic

In [5]:
data2016=make_sum(df,0,268)
data2017=make_sum(df,268,481)
data2018=make_sum(df,481,750)
data2019=make_sum(df,750,len(df)-1)

In [6]:
def make_sum2(dic):
    for tid in teamids:
        for vstid in teamids:
            if tid==vstid:
                continue
            else:
                if vstid not in dic[tid].keys():
                    continue
                for j in range(len(features)):
                    dic[tid][vstid][j+1]=dic[tid][vstid][j+1]/dic[tid][vstid][0]  #경기당 평균을 넣음
    return dic

In [7]:
data2016final=make_sum2(data2016)
data2017final=make_sum2(data2017)
data2018final=make_sum2(data2018)
data2019final=make_sum2(data2019)

In [8]:
from openpyxl import Workbook
wb = Workbook()

#파일 이름을 정하고, 데이터를 넣을 시트를 활성화합니다.
sheet1 = wb.active

#시트의 이름을 정합니다.
sheet1.title = 'sampdleSheet'

In [9]:
sheet1.append(['year','home','away','경기수','피장타율','방어율','삼진/볼넷','피출루율','피OPS','9이닝당 삼진수','9이닝당 볼넷수','잔루처리율'])

In [10]:
def make_csv(dic,year):
    for home in teamids:
        for away in teamids:
            if home==away:
                continue
            else:
                if away not in dic[home].keys():
                    continue
                else:
                    sheet1.append([year,home,away,dic[home][away][0],dic[home][away][1],dic[home][away][2],dic[home][away][3],dic[home][away][4],dic[home][away][5],dic[home][away][6],dic[home][away][7],dic[home][away][8],dic[home][away][9],dic[home][away][10],dic[home][away][11],dic[home][away][12],dic[home][away][13],dic[home][away][14],dic[home][away][15],dic[home][away][16],dic[home][away][17],dic[home][away][18],dic[home][away][19],dic[home][away][20],dic[home][away][21],dic[home][away][22],dic[home][away][23],dic[home][away][24],dic[home][away][25],dic[home][away][26],dic[home][away][27],dic[home][away][28],dic[home][away][29]])

In [11]:
make_csv(data2016final,2016)
make_csv(data2017final,2017)
make_csv(data2018final,2018)
make_csv(data2019final,2019)

In [12]:
wb.save('input_data_2.xlsx')